In [1]:
from hdf5libs import HDF5RawDataFile
import h5py

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
import daqdataformats
import detdataformats
from rawdatautils.unpack.wib2 import *
from rawdatautils.utilities.wib2 import *
import detchannelmaps

In [3]:
import time
import numpy as np
import numpy.ma as ma
import pandas as pd

In [4]:
import plotly.express as px
import plotly.graph_objects as go

In [5]:
def get_flattened_array(values):
    arr_flat = np.array([])
    for a in values:
        arr_flat = np.append(arr_flat,a)
    return arr_flat

In [6]:
filename='/Users/wketchum/Data/ProtoDUNE/np02_bde_coldbox_run021519_0418_dataflow0_datawriter_0_20230508T015309.hdf5.copied'
channel_map='VDColdboxChannelMap'
det='VD_Bottom_TPC'

In [7]:
with h5py.File(filename, 'r') as f:
    run_number = f.attrs["run_number"]

In [8]:
ch_map = detchannelmaps.make_map(channel_map)
offline_ch_num_dict = {}
offline_ch_plane_dict = {}

In [9]:
h5_file = HDF5RawDataFile(filename)

In [10]:
df_cols = ["run","record","sequence",
           "geoID","crate","slot","link","ichans",
           "trigger_timestamp","n_frames",
           "timestamps","adcs"]
df = pd.DataFrame(columns=df_cols)

In [11]:
records = h5_file.get_all_record_ids()

In [12]:
records_to_process = records[0:10]

In [13]:
for r in records_to_process:
    print(r)
    geo_ids = h5_file.get_geo_ids_for_subdetector(r,detdataformats.DetID.string_to_subdetector(det))

    for gid in geo_ids:
    
        df_gid_dict = dict.fromkeys(df_cols,None)
    
        df_gid_dict["run"] = run_number
        df_gid_dict["record"] = r[0]
        df_gid_dict["sequence"] = r[1]
        df_gid_dict["geoID"]=gid
    
        frag = h5_file.get_frag(r,gid)

        df_gid_dict["n_frames"]=get_n_frames(frag)
        df_gid_dict["trigger_timestamp"]=frag.get_trigger_timestamp()
    
        wf = detdataformats.wib2.WIB2Frame(frag.get_data())
        wh = wf.get_header()
    
        df_gid_dict["crate"]=wh.crate
        df_gid_dict["slot"]=wh.slot
        df_gid_dict["link"]=wh.link
        df_gid_dict["ichans"]=[np.arange(256)]

        df_gid_dict["timestamps"] = [np_array_timestamp(frag)]
    
    
        df_gid_dict["adcs"] = [np_array_adc(frag)]
        df = pd.concat([df,pd.DataFrame(data=df_gid_dict)])

(38457, 0)
(38458, 0)
(38459, 0)
(38460, 0)
(38461, 0)
(38462, 0)
(38463, 0)
(38464, 0)
(38465, 0)
(38466, 0)


In [14]:
df = df.set_index(["run","record","sequence","geoID"])

In [15]:
df["timestamps_diff"] = df["timestamps"].apply(np.diff)
df = df.join(df["timestamps_diff"].apply(lambda x: pd.Series(np.unique(x, return_counts=True),index=["timestamps_diff_vals","timestamps_diff_counts"])))

In [16]:
df["timestamps_first"] = df["timestamps"].apply(lambda x: x[0])

In [17]:
df["channels"] = df.apply(lambda x: [ch_map.get_offline_channel_from_crate_slot_fiber_chan(x.crate, x.slot, x.link, c) for c in x.ichans ], axis=1 )

In [18]:
df["plane"] = df.apply(lambda x: [ch_map.get_plane_from_offline_channel(uc) for uc in x.channels],axis=1)

In [19]:
df["adcs_rms"] = df["adcs"].apply(lambda x: np.std(x,axis=0))
df["adcs_ped"] = df["adcs"].apply(lambda x: np.mean(x,axis=0))

In [20]:
df

crate slot link   
run   record sequence geoID                             
21519 38457  0        8590327818          6    2    0  \
                      17180262410         6    4    0   
                      393226              6    0    0   
                      4295360522          6    1    0   
                      12885295114         6    3    0   
...                                     ...  ...  ...   
      38466  0        281479272071178     6    1    1   
                      281483567038474     6    2    1   
                      281487862005770     6    3    1   
                      281492156973066     6    4    1   
                      281496451940362     6    5    1   

                                                                                  ichans   
run   record sequence geoID                                                                
21519 38457  0        8590327818       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...  \
                      17180262410      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      393226           [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      4295360522       [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      12885295114      [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
...                                                                                  ...   
      38466  0        281479272071178  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      281483567038474  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      281487862005770  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      281492156973066  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                      281496451940362  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                        trigger_timestamp n_frames   
run   record sequence geoID                                          
21519 38457  0        8590327818       105219424107560482     8193  \
                      17180262410      105219424107560482     8193   
                      393226           105219424107560482     8193   
                      4295360522       105219424107560482     8193   
                      12885295114      105219424107560482     8193   
...                                                   ...      ...   
      38466  0        281479272071178  105219429734388007     8193   
                      281483567038474  105219429734388007     8193   
                      281487862005770  105219429734388007     8193   
                      281492156973066  105219429734388007     8193   
                      281496451940362  105219429734388007     8193   

                                                                              timestamps   
run   record sequence geoID                                                                
21519 38457  0        8590327818       [105219424107558315, 105219424107558347, 10521...  \
                      17180262410      [105219424107558315, 105219424107558347, 10521...   
                      393226           [105219424107558315, 105219424107558347, 10521...   
                      4295360522       [105219424107558315, 105219424107558347, 10521...   
                      12885295114      [105219424107558315, 105219424107558347, 10521...   
...                                                                                  ...   
      38466  0        281479272071178  [105219429734385835, 105219429734385867, 10521...   
                      281483567038474  [105219429734385835, 105219429734385867, 10521...   
                      281487862005770  [105219429734385835, 105219429734385867, 10521...   
                      281492156973066  [105219429734385835, 105219429734385867, 10521...   
                      281496451940362  [105219429734385835, 105219429734385867, 1052

In [21]:
df["timestamps_diff_OK"] = df["timestamps_diff_vals"].apply(lambda x: (len(x)==1 and x[0]==32))

In [23]:
fig = px.histogram(x=df.iloc[0]["timestamps_diff"],nbins=32*10,range_x=[0,32*10],range_y=[0.1,1e4],log_y=True)
fig.show()

In [24]:
im_timestamps_diff_OK = np.full((4,6),-1)
im_timestamps_diff_OK_txt = np.full((4,6),"???")
for i, row in df.iterrows():
    im_timestamps_diff_OK[row["link"]][row["slot"]] = 1 if row["timestamps_diff_OK"] else 0
    im_timestamps_diff_OK_txt[row["link"]][row["slot"]] = b"OK" if row["timestamps_diff_OK"] else b"NOT OK"

In [25]:
fig = go.Figure(data=go.Heatmap(z=im_timestamps_diff_OK,text=im_timestamps_diff_OK_txt,texttemplate="%{text}",colorscale='reds_r',zmax=1,zmin=-1))
fig.show()

In [26]:
arr_rms = get_flattened_array(df["adcs_rms"].values)
arr_chans = get_flattened_array(df["channels"].values)
arr_plane = get_flattened_array(df["plane"].values)

In [27]:
arr_plane==1

array([ True, False,  True, ..., False, False, False])

In [28]:
fig = px.scatter(x=ma.array(arr_chans,mask=(arr_plane!=1)),y=ma.array(arr_rms,mask=(arr_plane!=1)))
fig.show()